In [1]:
import geopandas as gpd 
import os 
import rasterio
import scipy.sparse as sparse 
import numpy as np
import pandas as pd


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# 2006 dataset has a shapefile and a csv file
#The coordinates will be extracted fom the shapefile while the population values will be obtained from the csv file

In [3]:
df = pd.read_csv('GEOSTAT_grid_EU_POP_2006_1K_V1_1_1.csv', sep=';')
df_2006 = df[['GRD_ID', 'POP_TOT']]

In [4]:
df_2006.head()

,GRD_ID,POP_TOT
0,1kmN5142E2862,2
1,1kmN5141E2862,13
2,1kmN5141E2864,211
3,1kmN5140E2862,1
4,1kmN5139E2876,33


In [5]:
# Read the points shapefile using GeoPandas 
stations_2006 = gpd.read_file(r'Grid_ETRS89_LAEA_1K_ref_GEOSTAT_2006.shp')

In [6]:
# Create a GeoDataFrame
gdf_2006 = gpd.GeoDataFrame(stations_2006, geometry='geometry', crs='EPSG:3035')

# Define the original coordinate system (UTM)
original_crs = {'init': 'epsg:3035'}  # Replace with the correct EPSG code for your data

# Transform the geometry to latitude and longitude
gdf_2006.to_crs(epsg=4326, inplace=True)  # 4326 is the EPSG code for WGS84 (latitude and longitude)

# Extract latitude and longitude from the transformed geometry
gdf_2006['latitude'] = gdf_2006['geometry'].centroid.y
gdf_2006['longitude'] = gdf_2006['geometry'].centroid.x

/var/folders/3x/53s4q5nj6516q9d1lrgjzpsr0000gn/T/ipykernel_20960/1812756185.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_2006['latitude'] = gdf_2006['geometry'].centroid.y
/var/folders/3x/53s4q5nj6516q9d1lrgjzpsr0000gn/T/ipykernel_20960/1812756185.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_2006['longitude'] = gdf_2006['geometry'].centroid.x


In [7]:
gdf_2006 = gdf_2006[['GRD_INSPIR', 'latitude', 'longitude']]

In [8]:
gdf_2006.head()

,GRD_INSPIR,latitude,longitude
0,1kmN1760E2636,37.025516,-8.991978
1,1kmN1954E2636,38.723753,-9.483677
2,1kmN1960E2636,38.776153,-9.499447
3,1kmN1961E2636,38.784885,-9.502079
4,1kmN1760E2637,37.027749,-8.981059


The 2021 Population folder has a .gpkg file which contains coordinates and raster data

In [65]:
#Read the .gpkg file

stations_2021 = gpd.read_file(r'ESTAT_Census_2021_V1-0.gpkg')

In [67]:
# Create a GeoDataFrame FOR 2021
gdf_2021 = gpd.GeoDataFrame(stations_2021, geometry='geometry', crs='EPSG:3035')

# Define the original coordinate system (UTM)
original_crs = {'init': 'epsg:3035'}  # Replace with the correct EPSG code for your data

# Transform the geometry to latitude and longitude
gdf_2021.to_crs(epsg=4326, inplace=True)  # 4326 is the EPSG code for WGS84 (latitude and longitude)

# Extract latitude and longitude from the transformed geometry
gdf_2021['latitude'] = gdf_2021['geometry'].centroid.y
gdf_2021['longitude'] = gdf_2021['geometry'].centroid.x

# Display the resulting GeoDataFrame
#print(gdf[['GRD_ID', 'latitude', 'longitude']])

/var/folders/3x/53s4q5nj6516q9d1lrgjzpsr0000gn/T/ipykernel_20960/3435711038.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_2021['latitude'] = gdf_2021['geometry'].centroid.y
/var/folders/3x/53s4q5nj6516q9d1lrgjzpsr0000gn/T/ipykernel_20960/3435711038.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_2021['longitude'] = gdf_2021['geometry'].centroid.x


In [68]:
gdf_2021.head()

,GRD_ID,OBS_VALUE_T,geometry,latitude,longitude
0,CRS3035RES1000mN1000000E1964000,0.0,"POLYGON ((-14.03656 28.68012, -14.02678 28.682...",28.685878,-14.032916
1,CRS3035RES1000mN1000000E1965000,0.0,"POLYGON ((-14.02678 28.68290, -14.01701 28.685...",28.688652,-14.023142
2,CRS3035RES1000mN1000000E1966000,118.0,"POLYGON ((-14.01701 28.68567, -14.00724 28.688...",28.691425,-14.013368
3,CRS3035RES1000mN1000000E1967000,4.0,"POLYGON ((-14.00724 28.68844, -13.99746 28.691...",28.694196,-14.003593
4,CRS3035RES1000mN1000000E1968000,0.0,"POLYGON ((-13.99746 28.69121, -13.98769 28.693...",28.696967,-13.993817


In [69]:
gdf_2021 = gdf_2021[['GRD_ID', 'OBS_VALUE_T', 'latitude', 'longitude']]

In [70]:
gdf_2021.head()

,GRD_ID,OBS_VALUE_T,latitude,longitude
0,CRS3035RES1000mN1000000E1964000,0.0,28.685878,-14.032916
1,CRS3035RES1000mN1000000E1965000,0.0,28.688652,-14.023142
2,CRS3035RES1000mN1000000E1966000,118.0,28.691425,-14.013368
3,CRS3035RES1000mN1000000E1967000,4.0,28.694196,-14.003593
4,CRS3035RES1000mN1000000E1968000,0.0,28.696967,-13.993817


In [71]:
gdf_2021.dtypes

GRD_ID          object
OBS_VALUE_T    float64
latitude       float64
longitude      float64
dtype: object

In [72]:
def dms_to_decimal(degrees, minutes, seconds, direction):
    sign = 1 if direction in ["N", "E"] else -1
    decimal_degrees = degrees + minutes / 60 + seconds / 3600
    return sign * decimal_degrees

Geographic Bounding Box for Slovakia

northernmost point: Oravská Polhora, 49°36′49.43′′N 19°28′2.52′′E 
southernmost point: Patince, 47°44′1.29′′N 18°17′20.93′′E 
westernmost point: Záhorská Ves, 48°22′50.24′′N 16°50′0.74′′E 
easternmost point: Nová Sedlica, 49°5′16.81′′N 22°33′56.7′′E

In [73]:
# Define the coordinates of the Slovakia bounding box

north_lat = dms_to_decimal(49, 36, 49.43, "N") # Oravská Polhora latitude
north_lon = dms_to_decimal(19, 28, 2.52, "E") # Oravská Polhora longitude
south_lat = dms_to_decimal(47, 44, 1.29, "N") # Patince latitude
south_lon = dms_to_decimal(18, 17, 20.93, "E") #Patince longitude
west_lat = dms_to_decimal(48, 22, 50.24, "N")# Záhorská Ves latitude
west_lon = dms_to_decimal(16, 50, 0.74, "E") # Záhorská Ves longitude
east_lat = dms_to_decimal(49, 5, 16.81, "E") # Nová Sedlica latitude
east_lon = dms_to_decimal(22, 33, 56.7, "E")   # Nová Sedlica longitude

In [74]:
print(north_lat)
print(north_lon)
print(south_lat)
print(south_lon)
print(west_lat)
print(west_lon)
print(east_lat)
print(east_lon)

49.613730555555556
19.467366666666663
47.733691666666665
18.289147222222223
48.38062222222222
16.83353888888889
49.08800277777778
22.56575


In [75]:
from shapely.geometry import box  # Import box from Shapely

# Create a GeoDataFrame with the bounding box polygon
bbox = gpd.GeoDataFrame(
    geometry=[box(west_lon, south_lat, east_lon, north_lat)],
    crs="EPSG:4326"  # Coordinate Reference System: WGS84
)


# Filter the coordinates within the bounding box
slovakia_data = gpd.clip(stations_2006, bbox)


In [76]:
len(slovakia_data)

29341

In [77]:
slovakia_data.head()

,GRD_INSPIR,geometry,latitude,longitude
1479147,1kmN2764E4889,"POLYGON ((17.58342 47.73621, 17.59669 47.73529...",47.731280,17.589394
1478508,1kmN2764E4888,"POLYGON ((17.57015 47.73713, 17.58342 47.73621...",47.732202,17.576126
1476595,1kmN2764E4885,"POLYGON ((17.53035 47.73989, 17.54362 47.73897...",47.734956,17.536322
1475932,1kmN2764E4884,"POLYGON ((17.51708 47.74080, 17.53035 47.73989...",47.735871,17.523053
1491907,1kmN2766E4908,"POLYGON ((17.83818 47.73630, 17.85145 47.73535...",47.731351,17.844130


In [78]:
# Merge DataFrames based on columns 'latitude' and 'longitude'
merged_slovakia_df = pd.merge(slovakia_data, gdf_2021, on=['latitude', 'longitude'])

In [79]:
len(merged_slovakia_df)

21383

In [80]:
merged_slovakia_df.head()

,GRD_INSPIR,geometry,latitude,longitude,GRD_ID,OBS_VALUE_T
0,1kmN2764E4889,"POLYGON ((17.58342 47.73621, 17.59669 47.73529...",47.731280,17.589394,CRS3035RES1000mN2764000E4889000,232.0
1,1kmN2764E4888,"POLYGON ((17.57015 47.73713, 17.58342 47.73621...",47.732202,17.576126,CRS3035RES1000mN2764000E4888000,266.0
2,1kmN2764E4885,"POLYGON ((17.53035 47.73989, 17.54362 47.73897...",47.734956,17.536322,CRS3035RES1000mN2764000E4885000,0.0
3,1kmN2764E4884,"POLYGON ((17.51708 47.74080, 17.53035 47.73989...",47.735871,17.523053,CRS3035RES1000mN2764000E4884000,550.0
4,1kmN2766E4908,"POLYGON ((17.83818 47.73630, 17.85145 47.73535...",47.731351,17.844130,CRS3035RES1000mN2766000E4908000,303.0


In [81]:
#Rename the GRID column in the csv to correspond for merging

df_2006 = df_2006.rename(columns= {'GRD_ID': 'GRD_INSPIR'})

In [82]:
df_2006.columns

Index(['GRD_INSPIR', 'POP_TOT'], dtype='object')

In [83]:
new_df = merged_slovakia_df[['GRD_INSPIR', 'GRD_ID', 'latitude', 'longitude', 'OBS_VALUE_T']]

In [84]:
#Finally merging slovakia region with the csv file that contains the population values for 2006.

final_df = pd.merge(new_df, df_2006, on=['GRD_INSPIR'])

In [85]:
len(final_df)

21383

In [86]:
final_df.head(100)

,GRD_INSPIR,GRD_ID,latitude,longitude,OBS_VALUE_T,POP_TOT
0,1kmN2764E4889,CRS3035RES1000mN2764000E4889000,47.731280,17.589394,232.0,1
1,1kmN2764E4888,CRS3035RES1000mN2764000E4888000,47.732202,17.576126,266.0,24
2,1kmN2764E4885,CRS3035RES1000mN2764000E4885000,47.734956,17.536322,0.0,1
3,1kmN2764E4884,CRS3035RES1000mN2764000E4884000,47.735871,17.523053,550.0,514
4,1kmN2766E4908,CRS3035RES1000mN2766000E4908000,47.731351,17.844130,303.0,462
5,1kmN2766E4907,CRS3035RES1000mN2766000E4907000,47.732303,17.830867,1686.0,1520
6,1kmN2766E4906,CRS3035RES1000mN2766000E4906000,47.733254,17.817603,651.0,640
7,1kmN2767E4907,CRS3035RES1000mN2767000E4907000,47.741245,17.832236,327.0,295
8,1kmN2767E4906,CRS3035RES1000mN2767000E4906000,47.742196,17.818970,0.0,29
9,1kmN2765E4901,CRS3035RES1000mN2765000E4901000,47.729041,17.749921,0.0,61


In [87]:
final_df = final_df.rename(columns= {'OBS_VALUE_T': '2021_POP', 'POP_TOT': "2006_POP"})


In [89]:
#Get the percentage increase in population between 2006 and 2021

final_df['Relative_increase'] = (final_df['2021_POP'] - final_df['2006_POP']) / final_df['2006_POP'] * 100

In [90]:
final_df.head()

,GRD_INSPIR,GRD_ID,latitude,longitude,2021_POP,2006_POP,Relative_increase
0,1kmN2764E4889,CRS3035RES1000mN2764000E4889000,47.731280,17.589394,232.0,1,23100.000000
1,1kmN2764E4888,CRS3035RES1000mN2764000E4888000,47.732202,17.576126,266.0,24,1008.333333
2,1kmN2764E4885,CRS3035RES1000mN2764000E4885000,47.734956,17.536322,0.0,1,-100.000000
3,1kmN2764E4884,CRS3035RES1000mN2764000E4884000,47.735871,17.523053,550.0,514,7.003891
4,1kmN2766E4908,CRS3035RES1000mN2766000E4908000,47.731351,17.844130,303.0,462,-34.415584


In [91]:
# Identify Top 5 Grid Cells with Highest Relative Increase

top_5_cells = final_df.nlargest(5, 'Relative_increase')

# Save the top 5 cells data
top_5_cells.to_csv('slovakia_top_5_areas.csv', index=False)


In [92]:
top_5_cells

,GRD_INSPIR,GRD_ID,latitude,longitude,2021_POP,2006_POP,Relative_increase
2108,1kmN2815E4857,CRS3035RES1000mN2815000E4857000,48.216395,17.229316,1059.0,3,35200.0
489,1kmN2771E4931,CRS3035RES1000mN2771000E4931000,47.753690,18.156191,341.0,1,34000.0
1141,1kmN2803E4859,CRS3035RES1000mN2803000E4859000,48.107256,17.240703,305.0,1,30400.0
2117,1kmN2816E4857,CRS3035RES1000mN2816000E4857000,48.225343,17.230599,1212.0,4,30200.0
3236,1kmN2814E4865,CRS3035RES1000mN2814000E4865000,48.200358,17.335189,466.0,2,23200.0


In [93]:
# Calculate Average and Median Population for Slovakia in 2021

average_population = final_df['2021_POP'].mean()
median_population = final_df['2021_POP'].median()

# Save the results
results = pd.DataFrame({'average_population': [average_population], 'median_population': [median_population]})
results.to_csv('slovakia_2021_population_stats.csv', index=False)


In [94]:
print(average_population)
print(median_population)

311.72001122386945
99.0
